In [79]:
import pandas as pd 
import seaborn as sns
import numpy as np
import pickle
from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec
import networkx as nx
import os
from tqdm.notebook import tqdm
# import warnings
# warnings.filterwarnings('ignore')

import multiprocessing as mp
from multiprocessing import cpu_count
import pickle
#mp.set_start_method('spawn')
from myfunctions import extract_freqs, calc_iter

We have some set of datasets

In [2]:
path_to_datasets = '../datasets/'
path_to_uniform_data = '../data/'
path_to_output = '../output/'
path_to_samples = '../samples/'

functions

In [54]:
def add_size_n_dens(mdf):
    path_to_gph = '/Users/zaikoval/Downloads/graphs_5types/graphs'
    for idx in mdf.index:
        graph = nx.from_pandas_edgelist(pd.read_csv(os.path.join(path_to_gph, '-'+str(idx)+'.csv'), names=['source', 'target']))
        n = graph.number_of_nodes()
        e = graph.number_of_edges()
        mdf.loc[idx, 'nodes'] = n
        mdf.loc[idx, 'edges'] = e
        mdf.loc[idx, 'density'] = 2*e/(n*(n-1))

## Reading datasets

In [126]:
def read_graph_data(filename, sep=',', header=None, skiprows=0):
    edgelist = pd.read_csv(os.path.join(path_to_datasets, filename), sep=sep, skiprows=skiprows, header=header, names=['source', 'target'])
    edgelist.to_csv(os.path.join(path_to_uniform_data, filename), index=False, header=None)
    return nx.from_pandas_edgelist(edgelist)

In [127]:
citeseer = read_graph_data('citeseer.cites', sep='\t')
deezer_europe = read_graph_data('deezer_europe_edges.csv', header=0)
lastfm_asia = read_graph_data('lastfm_asia_edges.csv', header=0)
cora = read_graph_data('cora.cites', sep='\t')
email_Eu_core = read_graph_data('email-Eu-core.txt', sep=' ')
fb_0 = read_graph_data('0.edges', sep=' ')
fb_1 = read_graph_data('107.edges', sep=' ')
fb_2 = read_graph_data('348.edges', sep=' ')
fb_3 = read_graph_data('414.edges', sep=' ')
fb_4 = read_graph_data('686.edges', sep=' ')
fb_5 = read_graph_data('698.edges', sep=' ')
fb_6 = read_graph_data('1684.edges', sep=' ')
fb_7 = read_graph_data('1912.edges', sep=' ')
fb_8 = read_graph_data('3437.edges', sep=' ')
fb_9 = read_graph_data('3980.edges', sep=' ')
email_univ = read_graph_data('email-univ.edges', sep=' ')
fb_company = read_graph_data('fb-pages-company.edges', header=0)
fb_food = read_graph_data('fb-pages-food.edges')
fb_politician = read_graph_data('fb-pages-politician.edges')
fb_public_figure = read_graph_data('fb-pages-politician.edges')
fb_tvshow = read_graph_data('fb-pages-tvshow.edges')
soc_anybeat = read_graph_data('soc-anybeat.edges', sep=' ')
soc_hamsterster = read_graph_data('soc-hamsterster.edges', sep=' ', skiprows=2)
soc_wiki_vote = read_graph_data('soc-wiki-Vote.mtx', sep=' ', skiprows=2)
cit_DBLP = read_graph_data('cit-DBLP.edges', sep=' ', skiprows=2)

# list_of_graphs = [citeseer, deezer_europe, lastfm_asia, cora, email_Eu_core,
#                   fb_0, fb_1, fb_2, fb_3, fb_4, fb_5, fb_6, fb_7, fb_8, fb_9,
#                   email_univ, fb_company, fb_food, fb_politician, fb_public_figure,
#                   fb_tvshow, soc_anybeat, soc_hamsterster, soc_wiki_vote, cit_DBLP]
# len(list_of_graphs)

25

## Graph classification dataset for benchmark

In [10]:
%cd ../datasets/
!wget http://nrvis.com/download/data/labeled/REDDIT-BINARY.zip
!unzip REDDIT-BINARY.zip
%cd ../notebooks/

/Users/zaikoval/Documents/GitHub/regression-on-sampling/datasets
--2020-08-30 16:02:21--  http://nrvis.com/download/data/labeled/REDDIT-BINARY.zip
Распознаётся nrvis.com (nrvis.com)… 173.236.156.25
Подключение к nrvis.com (nrvis.com)|173.236.156.25|:80... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 9924710 (9,5M) [application/zip]
Сохранение в: «REDDIT-BINARY.zip»

REDDIT-BINARY.zip   100%[===================>]   9,46M  2,08MB/s    за 7,7s    

2020-08-30 16:02:29 (1,23 MB/s) - «REDDIT-BINARY.zip» сохранён [9924710/9924710]

Archive:  REDDIT-BINARY.zip
  inflating: REDDIT-BINARY.edges     
  inflating: readme.html             
  inflating: REDDIT-BINARY.graph_idx  
  inflating: REDDIT-BINARY.graph_labels  
  inflating: REDDIT-BINARY.readme    
/Users/zaikoval/Documents/GitHub/regression-on-sampling/notebooks


In [11]:
edges = pd.read_csv(os.path.join(path_to_datasets, 'REDDIT-BINARY.edges'), header=None, names=['source', 'target'])

In [12]:
full_graph = nx.from_pandas_edgelist(edges)

In [13]:
graph_idx = pd.read_csv(os.path.join(path_to_datasets, 'REDDIT-BINARY.graph_idx'), header=None)

In [14]:
graphs = list()
for i in tqdm(graph_idx[0].unique()):
    set_of_nodes = graph_idx[graph_idx[0] == i].index + 1
    subgraph = nx.subgraph(full_graph, set_of_nodes)
    connected_subgraph = nx.subgraph(full_graph, max(nx.connected_components(subgraph), key=len)) #extract max connected subgraph
    graphs.append(connected_subgraph)

In [15]:
graphs = graphs[:200]

In [13]:
#считаем ndlib for them 1h40m
iterations = []
for elem in tqdm(graphs):
    iterations.append(calc_iter(elem))

/home/zaikoval/anaconda3/lib/python3.7/site-packages/ndlib/models/DiffusionModel.py:170: UserWarning: The fraction_infected value is too low given the number of nodes of the selected graph: a single node will be set as infected
  "The fraction_infected value is too low given the number of nodes of the selected graph: a "


KeyboardInterrupt: 

### Переписать функцию с параметром parallelized

In [20]:
iterations = [] 

with mp.Pool(cpu_count()) as p:
    iterations = list(tqdm(p.imap(calc_iter, graphs, chunksize=6), total=len(graphs)))

/home/zaikoval/anaconda3/lib/python3.7/site-packages/ndlib/models/DiffusionModel.py:170: UserWarning: The fraction_infected value is too low given the number of nodes of the selected graph: a single node will be set as infected
  "The fraction_infected value is too low given the number of nodes of the selected graph: a "
/home/zaikoval/anaconda3/lib/python3.7/site-packages/ndlib/models/DiffusionModel.py:170: UserWarning: The fraction_infected value is too low given the number of nodes of the selected graph: a single node will be set as infected
  "The fraction_infected value is too low given the number of nodes of the selected graph: a "
/home/zaikoval/anaconda3/lib/python3.7/site-packages/ndlib/models/DiffusionModel.py:170: UserWarning: The fraction_infected value is too low given the number of nodes of the selected graph: a single node will be set as infected
  "The fraction_infected value is too low given the number of nodes of the selected graph: a "


In [23]:
pd.DataFrame(iterations).to_csv(os.path.join(path_to_output, 'ndlib/ndlib.csv'), index=False, header=None)

In [20]:
ndl_data = pd.read_csv(os.path.join(path_to_output, 'ndlib/ndlib.csv'), header=None)
ndl_data

,0
0,32.9
1,40.2
2,45.9
3,38.3
4,41.6
...,...
195,38.0
196,47.4
197,38.3
198,45.7


In [ ]:
for elem in tqdm(graphs): #считам motif distribution for them

In [50]:
freqs = []
for item in tqdm(graphs):
    freqs.append(extract_freqs(item, size=4))

------------------------------------------
gtrieScanner (version 0.1)
------------------------------------------
Subgraph Size: 4
Graph File: /Users/zaikoval/Documents/GitHub/regression-on-sampling/output/temp/temp.txt
Undirected, 214 Nodes, 238 Edges
Method:     GTRIE with file containing complete g-trie

Counting subgraph frequency on 'ORIGINAL NETWORK'
Reading G-Trie
Creation time: 0.00
Compression rate = 58.33%

Census time: 0.00
5 subgraphs, 350598 occurrences
Time elapsed: 0.001768s

Results written to file "../output/temp.html"

------------------------------------------
gtrieScanner (version 0.1)
------------------------------------------
Subgraph Size: 4
Graph File: /Users/zaikoval/Documents/GitHub/regression-on-sampling/output/temp/temp.txt
Undirected, 695 Nodes, 817 Edges
Method:     GTRIE with file containing complete g-trie

Counting subgraph frequency on 'ORIGINAL NETWORK'
Reading G-Trie
Creation time: 0.00
Compression rate = 58.33%

Census time: 0.03
5 subgraphs, 6617309

------------------------------------------
gtrieScanner (version 0.1)
------------------------------------------
Subgraph Size: 4
Graph File: /Users/zaikoval/Documents/GitHub/regression-on-sampling/output/temp/temp.txt
Undirected, 894 Nodes, 1150 Edges
Method:     GTRIE with file containing complete g-trie

Counting subgraph frequency on 'ORIGINAL NETWORK'
Reading G-Trie
Creation time: 0.00
Compression rate = 58.33%

Census time: 0.04
6 subgraphs, 9084325 occurrences
Time elapsed: 0.039945s

Results written to file "../output/temp.html"

------------------------------------------
gtrieScanner (version 0.1)
------------------------------------------
Subgraph Size: 4
Graph File: /Users/zaikoval/Documents/GitHub/regression-on-sampling/output/temp/temp.txt
Undirected, 1442 Nodes, 1713 Edges
Method:     GTRIE with file containing complete g-trie

Counting subgraph frequency on 'ORIGINAL NETWORK'
Reading G-Trie
Creation time: 0.00
Compression rate = 58.33%

Census time: 0.33
6 subgraphs, 589

------------------------------------------
gtrieScanner (version 0.1)
------------------------------------------
Subgraph Size: 4
Graph File: /Users/zaikoval/Documents/GitHub/regression-on-sampling/output/temp/temp.txt
Undirected, 1161 Nodes, 1351 Edges
Method:     GTRIE with file containing complete g-trie

Counting subgraph frequency on 'ORIGINAL NETWORK'
Reading G-Trie
Creation time: 0.00
Compression rate = 58.33%

Census time: 0.22
6 subgraphs, 42295661 occurrences
Time elapsed: 0.224598s

Results written to file "../output/temp.html"

------------------------------------------
gtrieScanner (version 0.1)
------------------------------------------
Subgraph Size: 4
Graph File: /Users/zaikoval/Documents/GitHub/regression-on-sampling/output/temp/temp.txt
Undirected, 734 Nodes, 829 Edges
Method:     GTRIE with file containing complete g-trie

Counting subgraph frequency on 'ORIGINAL NETWORK'
Reading G-Trie
Creation time: 0.00
Compression rate = 58.33%

Census time: 0.06
5 subgraphs, 131

------------------------------------------
gtrieScanner (version 0.1)
------------------------------------------
Subgraph Size: 4
Graph File: /Users/zaikoval/Documents/GitHub/regression-on-sampling/output/temp/temp.txt
Undirected, 1879 Nodes, 2323 Edges
Method:     GTRIE with file containing complete g-trie

Counting subgraph frequency on 'ORIGINAL NETWORK'
Reading G-Trie
Creation time: 0.00
Compression rate = 58.33%

Census time: 1.15
5 subgraphs, 180099644 occurrences
Time elapsed: 1.148270s

Results written to file "../output/temp.html"

------------------------------------------
gtrieScanner (version 0.1)
------------------------------------------
Subgraph Size: 4
Graph File: /Users/zaikoval/Documents/GitHub/regression-on-sampling/output/temp/temp.txt
Undirected, 679 Nodes, 869 Edges
Method:     GTRIE with file containing complete g-trie

Counting subgraph frequency on 'ORIGINAL NETWORK'
Reading G-Trie
Creation time: 0.00
Compression rate = 58.33%

Census time: 0.01
5 subgraphs, 25

------------------------------------------
gtrieScanner (version 0.1)
------------------------------------------
Subgraph Size: 4
Graph File: /Users/zaikoval/Documents/GitHub/regression-on-sampling/output/temp/temp.txt
Undirected, 56 Nodes, 58 Edges
Method:     GTRIE with file containing complete g-trie

Counting subgraph frequency on 'ORIGINAL NETWORK'
Reading G-Trie
Creation time: 0.00
Compression rate = 58.33%

Census time: 0.00
3 subgraphs, 4509 occurrences
Time elapsed: 0.000334s

Results written to file "../output/temp.html"

------------------------------------------
gtrieScanner (version 0.1)
------------------------------------------
Subgraph Size: 4
Graph File: /Users/zaikoval/Documents/GitHub/regression-on-sampling/output/temp/temp.txt
Undirected, 182 Nodes, 196 Edges
Method:     GTRIE with file containing complete g-trie

Counting subgraph frequency on 'ORIGINAL NETWORK'
Reading G-Trie
Creation time: 0.00
Compression rate = 58.33%

Census time: 0.00
5 subgraphs, 85179 occur

...

------------------------------------------
gtrieScanner (version 0.1)
------------------------------------------
Subgraph Size: 4
Graph File: /Users/zaikoval/Documents/GitHub/regression-on-sampling/output/temp/temp.txt
Undirected, 549 Nodes, 638 Edges
Method:     GTRIE with file containing complete g-trie

Counting subgraph frequency on 'ORIGINAL NETWORK'
Reading G-Trie
Creation time: 0.00
Compression rate = 58.33%

Census time: 0.01
5 subgraphs, 2563250 occurrences
Time elapsed: 0.011329s

Results written to file "../output/temp.html"

------------------------------------------
gtrieScanner (version 0.1)
------------------------------------------
Subgraph Size: 4
Graph File: /Users/zaikoval/Documents/GitHub/regression-on-sampling/output/temp/temp.txt
Undirected, 540 Nodes, 664 Edges
Method:     GTRIE with file containing complete g-trie

Counting subgraph frequency on 'ORIGINAL NETWORK'
Reading G-Trie
Creation time: 0.00
Compression rate = 58.33%

Census time: 0.01
6 subgraphs, 112049

------------------------------------------
gtrieScanner (version 0.1)
------------------------------------------
Subgraph Size: 4
Graph File: /Users/zaikoval/Documents/GitHub/regression-on-sampling/output/temp/temp.txt
Undirected, 421 Nodes, 527 Edges
Method:     GTRIE with file containing complete g-trie

Counting subgraph frequency on 'ORIGINAL NETWORK'
Reading G-Trie
Creation time: 0.00
Compression rate = 58.33%

Census time: 0.00
5 subgraphs, 660976 occurrences
Time elapsed: 0.003156s

Results written to file "../output/temp.html"

------------------------------------------
gtrieScanner (version 0.1)
------------------------------------------
Subgraph Size: 4
Graph File: /Users/zaikoval/Documents/GitHub/regression-on-sampling/output/temp/temp.txt
Undirected, 785 Nodes, 953 Edges
Method:     GTRIE with file containing complete g-trie

Counting subgraph frequency on 'ORIGINAL NETWORK'
Reading G-Trie
Creation time: 0.00
Compression rate = 58.33%

Census time: 0.03
5 subgraphs, 5754314

------------------------------------------
gtrieScanner (version 0.1)
------------------------------------------
Subgraph Size: 4
Graph File: /Users/zaikoval/Documents/GitHub/regression-on-sampling/output/temp/temp.txt
Undirected, 681 Nodes, 850 Edges
Method:     GTRIE with file containing complete g-trie

Counting subgraph frequency on 'ORIGINAL NETWORK'
Reading G-Trie
Creation time: 0.00
Compression rate = 58.33%

Census time: 0.02
6 subgraphs, 3453709 occurrences
Time elapsed: 0.017634s

Results written to file "../output/temp.html"

------------------------------------------
gtrieScanner (version 0.1)
------------------------------------------
Subgraph Size: 4
Graph File: /Users/zaikoval/Documents/GitHub/regression-on-sampling/output/temp/temp.txt
Undirected, 277 Nodes, 312 Edges
Method:     GTRIE with file containing complete g-trie

Counting subgraph frequency on 'ORIGINAL NETWORK'
Reading G-Trie
Creation time: 0.00
Compression rate = 58.33%

Census time: 0.00
5 subgraphs, 520247

------------------------------------------
gtrieScanner (version 0.1)
------------------------------------------
Subgraph Size: 4
Graph File: /Users/zaikoval/Documents/GitHub/regression-on-sampling/output/temp/temp.txt
Undirected, 652 Nodes, 865 Edges
Method:     GTRIE with file containing complete g-trie

Counting subgraph frequency on 'ORIGINAL NETWORK'
Reading G-Trie
Creation time: 0.00
Compression rate = 58.33%

Census time: 0.03
6 subgraphs, 4103719 occurrences
Time elapsed: 0.027969s

Results written to file "../output/temp.html"

------------------------------------------
gtrieScanner (version 0.1)
------------------------------------------
Subgraph Size: 4
Graph File: /Users/zaikoval/Documents/GitHub/regression-on-sampling/output/temp/temp.txt
Undirected, 138 Nodes, 174 Edges
Method:     GTRIE with file containing complete g-trie

Counting subgraph frequency on 'ORIGINAL NETWORK'
Reading G-Trie
Creation time: 0.00
Compression rate = 58.33%

Census time: 0.00
5 subgraphs, 22773 

------------------------------------------
gtrieScanner (version 0.1)
------------------------------------------
Subgraph Size: 4
Graph File: /Users/zaikoval/Documents/GitHub/regression-on-sampling/output/temp/temp.txt
Undirected, 643 Nodes, 786 Edges
Method:     GTRIE with file containing complete g-trie

Counting subgraph frequency on 'ORIGINAL NETWORK'
Reading G-Trie
Creation time: 0.00
Compression rate = 58.33%

Census time: 0.05
6 subgraphs, 9437409 occurrences
Time elapsed: 0.048488s

Results written to file "../output/temp.html"

------------------------------------------
gtrieScanner (version 0.1)
------------------------------------------
Subgraph Size: 4
Graph File: /Users/zaikoval/Documents/GitHub/regression-on-sampling/output/temp/temp.txt
Undirected, 260 Nodes, 313 Edges
Method:     GTRIE with file containing complete g-trie

Counting subgraph frequency on 'ORIGINAL NETWORK'
Reading G-Trie
Creation time: 0.00
Compression rate = 58.33%

Census time: 0.00
5 subgraphs, 336083

In [52]:
pd.DataFrame(freqs)

,0,1,2,3,4,5
0,0.023235,0.000034,0.974224,0.002493,0.000014,0.0
1,0.008805,0.000022,0.990187,0.000983,0.000003,0.0
2,0.037216,0.000047,0.956137,0.006533,0.000067,0.0
3,0.015102,0.000026,0.982784,0.002080,0.000009,0.0
4,0.019005,0.000057,0.979387,0.001545,0.000005,0.0
...,...,...,...,...,...,...
195,0.033012,0.000077,0.962028,0.004842,0.000040,0.0
196,0.244257,0.001288,0.748766,0.005635,0.000054,0.0
197,0.025558,0.000049,0.972453,0.001935,0.000005,0.0
198,0.026766,0.000060,0.971582,0.001587,0.000005,0.0


In [ ]:
iters_list = [] # считаем для каждого из них ndlib количество итераций до заражения
for elem in tqdm(graphs_list): # make it with multiprocessing
    iters_list.append((elem, calc_iter(elem)))

In [ ]:
for elem in tqdm(graphs_list): # считаем для них распределение мотивов
    gt.extract_motifs(elem, 4, path_to_graphs=path_to_uniform_data, path_to_output=path_to_output)

In [10]:
for graph in tqdm(graphs_list): # сэмплируем из них 10 графов с примерно половиной количества нод
    
    G = nx.from_pandas_edgelist(
        pd.read_csv(os.path.join(path_to_uniform_data, graph), names=['source', 'target']))
    
    for j in range(20):
        extra_hop = set()
        first_node = np.random.choice(G.nodes()) # not just random, but only among low degree
        
        extra_hop = extra_hop.union(list(nx.neighbors(G, first_node)))
        i=1

        while (i<4) and (len(extra_hop)<G.number_of_nodes()):
            i+=1
            for node in extra_hop:
                 extra_hop = extra_hop.union(nx.neighbors(G, node))

            nx.to_pandas_edgelist(nx.subgraph(G, extra_hop)).to_csv(
            os.path.join(path_to_samples, graph+'_h{}_s{}.csv'.format(i,j+1)),
            header=None,
            index=False)

In [ ]:
# для каждого из них считаем распределения и усредняем

In [ ]:
# создаем датасет и делаем кроссвалидацию